### 前向传播、反向传播和计算图
梯度的自动计算（自动微分）大大简化了深度学习算法的实现。在自动微分之前，即使是对复杂模型的微小调整也需要手工重新计算复杂的导数，学术论文也不得不分配大量页面来推导更新规则。本节将通过一些基本的数学和计算图，深入探讨*反向传播*的细节。**首先，我们将重点放在带权重衰减（$L_2$正则化）的单隐藏层多层感知机上。**


### 前向传播
**前向传播（forward propagation或forward pass）指的是**：按顺序（从输入层到输出层）计算和存储神经网络中每层的结果。

我们将一步步研究单隐藏层神经网络的机制，为了简单起见，我们假设输入样本是 $\mathbf{x}\in \mathbb{R}^d$，**并且我们的隐藏层不包括偏置项**。这里的中间变量是：

$$\mathbf{z}= \mathbf{W}^{(1)} \mathbf{x},$$

其中$\mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$是隐藏层的权重参数。将中间变量$\mathbf{z}\in \mathbb{R}^h$通过激活函数$\phi$后，我们得到长度为$h$的隐藏激活向量：

$$\mathbf{h}= \phi (\mathbf{z}).$$

隐藏变量$\mathbf{h}$也是一个中间变量。假设输出层的参数只有权重$\mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$，我们可以得到输出层变量，它是一个长度为$q$的向量：

$$\mathbf{o}= \mathbf{W}^{(2)} \mathbf{h}.$$

假设损失函数为$l$，样本标签为$y$，我们可以计算单个数据样本的损失项，

$$L = l(\mathbf{o}, y).$$

根据$L_2$正则化的定义，给定超参数$\lambda$，正则化项为

**$$s = \frac{\lambda}{2} \left(\|\mathbf{W}^{(1)}\|_F^2 + \|\mathbf{W}^{(2)}\|_F^2\right),$$**

其中矩阵的Frobenius范数是将矩阵展平为向量后应用的$L_2$范数。**最后**，模型在给定数据样本上的正则化损失为：

$$J = L + s.$$

在下面的讨论中，我们将$J$称为**目标函数（objective function）。**


### 前向传播计算图

绘制**计算图**有助于我们可视化计算中操作符和变量的依赖关系。下图是与上述简单网络相对应的计算图，**其中正方形表示变量，圆圈表示操作符。左下角表示输入，右上角表示输出。注意显示数据流的箭头方向主要是向右和向上的**。

![前向传播的计算图](img/forward.svg)

### 反向传播

**反向传播（backward propagation或backpropagation）指的是计算神经网络参数梯度的方法。**简言之，该方法根据微积分中的**链式规则**，按**相反**的顺序从输出层到输入层遍历网络。**该算法存储了计算某些参数梯度时所需的任何中间变量（偏导数）。**假设我们有函数$\mathsf{Y}=f(\mathsf{X})$和$\mathsf{Z}=g(\mathsf{Y})$，其中输入和输出$\mathsf{X}, \mathsf{Y}, \mathsf{Z}$是任意形状的张量。利用链式法则，我们可以计算$\mathsf{Z}$关于$\mathsf{X}$的导数

$$\frac{\partial \mathsf{Z}}{\partial \mathsf{X}} = \text{prod}\left(\frac{\partial \mathsf{Z}}{\partial \mathsf{Y}}, \frac{\partial \mathsf{Y}}{\partial \mathsf{X}}\right).$$

在这里，**我们使用$\text{prod}$运算符在执行必要的操作（如换位和交换输入位置）后将其参数相乘**。对于向量，这很简单，它只是矩阵-矩阵乘法。对于高维张量，我们使用适当的对应项。**运算符$\text{prod}$指代了所有的这些符号**。

回想一下，在上述计算图中的单隐藏层简单网络的参数是$\mathbf{W}^{(1)}$和$\mathbf{W}^{(2)}$。反向传播的目的是计算梯度$\partial J/\partial \mathbf{W}^{(1)}$和$\partial J/\partial \mathbf{W}^{(2)}$。为此，我们应用链式法则，依次计算每个中间变量和参数的梯度。计算的顺序与前向传播中执行的顺序相反，因为我们需要从计算图的结果开始，并朝着参数的方向努力。第一步是计算目标函数$J=L+s$相对于损失项$L$和正则项$s$的梯度。

$$\frac{\partial J}{\partial L} = 1 \; \text{and} \; \frac{\partial J}{\partial s} = 1.$$

接下来，我们根据链式法则计算目标函数关于输出层变量$\mathbf{o}$的梯度：

$$
\frac{\partial J}{\partial \mathbf{o}}
= \text{prod}\left(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \mathbf{o}}\right)
= \frac{\partial L}{\partial \mathbf{o}}
\in \mathbb{R}^q.
$$

接下来，我们计算正则化项相对于两个参数的梯度：

$$\frac{\partial s}{\partial \mathbf{W}^{(1)}} = \lambda \mathbf{W}^{(1)}
\; \text{and} \;
\frac{\partial s}{\partial \mathbf{W}^{(2)}} = \lambda \mathbf{W}^{(2)}.$$

现在我们可以计算最接近输出层的模型参数的梯度
$\partial J/\partial \mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$。
使用链式法则得出：

$$\frac{\partial J}{\partial \mathbf{W}^{(2)}}= \text{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{W}^{(2)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(2)}}\right)= \frac{\partial J}{\partial \mathbf{o}} \mathbf{h}^\top + \lambda \mathbf{W}^{(2)}.$$

为了获得关于$\mathbf{W}^{(1)}$的梯度，我们需要继续沿着输出层到隐藏层反向传播。关于隐藏层输出的梯度$\partial J/\partial \mathbf{h} \in \mathbb{R}^h$由下式给出：

$$
\frac{\partial J}{\partial \mathbf{h}}
= \text{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{h}}\right)
= {\mathbf{W}^{(2)}}^\top \frac{\partial J}{\partial \mathbf{o}}.
$$

由于激活函数$\phi$是按元素计算的，计算中间变量$\mathbf{z}$的梯度$\partial J/\partial \mathbf{z} \in \mathbb{R}^h$需要使用**按元素乘法运算符，我们用$\odot$表示：**

$$
\frac{\partial J}{\partial \mathbf{z}}
= \text{prod}\left(\frac{\partial J}{\partial \mathbf{h}}, \frac{\partial \mathbf{h}}{\partial \mathbf{z}}\right)
= \frac{\partial J}{\partial \mathbf{h}} \odot \phi'\left(\mathbf{z}\right).
$$

最后，我们可以得到最接近输入层的模型参数的梯度$\partial J/\partial \mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$。根据链式法则，我们得到：

$$
\frac{\partial J}{\partial \mathbf{W}^{(1)}}
= \text{prod}\left(\frac{\partial J}{\partial \mathbf{z}}, \frac{\partial \mathbf{z}}{\partial \mathbf{W}^{(1)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(1)}}\right)
= \frac{\partial J}{\partial \mathbf{z}} \mathbf{x}^\top + \lambda \mathbf{W}^{(1)}.
$$

### 训练神经网络

**在训练神经网络时，前向传播和反向传播相互依赖**。对于前向传播，我们沿着依赖的方向遍历计算图并计算其路径上的所有变量。然后将这些用于反向传播，其中计算顺序与计算图的相反。

以上述简单网络为例：**一方面**，在前向传播期间计算正则项取决于模型参数$\mathbf{W}^{(1)}$和$\mathbf{W}^{(2)}$的当前值。它们是由优化算法根据最近迭代的反向传播给出的。**另一方面**，反向传播期间参数的梯度计算，取决于由前向传播给出的隐藏变量$\mathbf{h}$的当前值。

因此，在训练神经网络时，**在初始化模型参数后，我们交替使用前向传播和反向传播，利用反向传播给出的梯度来更新模型参数**。注意，反向传播**重复利用**前向传播中存储的中间值，以避免重复计算。**带来的影响之一是我们需要保留中间值，直到反向传播完成**。这也是训练比单纯的预测**需要更多的内存（显存）的原因之一**。此外，**这些中间值的大小与网络层的数量和批量的大小大致成正比**。因此，**使用更大的批量来训练更深层次的网络更容易导致内存不足（out of memory）错误。**

### 小结

* 前向传播在神经网络定义的计算图中按顺序计算和存储中间变量，它的顺序是从输入层到输出层。
* 反向传播按相反的顺序（从输出层到输入层）计算和存储神经网络的**中间变量和参数的梯度**。
* 在训练深度学习模型时，前向传播和反向传播是相互依赖的。
* **训练比预测需要更多的内存**。

--------------------------------------------

**将计算图划分到多个GPU上是一种常见的策略，可以加速模型的训练**。这个过程通常称为**模型的分布式训练**。下面是一个大致的思路，说明如何**将计算图划分到多个GPU上**：

1. **数据并行ism：** 将数据分成小批量，在每个GPU上复制一份计算图（包括网络层）和一批数据，然后在每个GPU上分别进行前向传播和反向传播。在反向传播完成后，收集各个GPU上的梯度并进行平均，然后更新模型的参数。

2. **模型并行ism：** 将网络的不同层分别放在不同的GPU上，然后在每个GPU上独立地进行前向传播和反向传播。在反向传播完成后，收集各个GPU上的梯度并进行平均，然后更新模型的参数。

3. **混合并行ism：** 结合数据并行ism和模型并行ism，将模型的某些层划分到不同的GPU上，同时在每个GPU上使用多个小批量进行训练。

在将计算图划分到多个GPU上时，需要考虑数据同步、梯度收集和更新等问题，这可能需要使用专门的分布式训练框架或库，例如 PyTorch 的 `torch.nn.DataParallel` 或 `torch.nn.parallel.DistributedDataParallel`。

**关于小批量训练与分布式训练的优缺点：**

**优点：**

1. **训练速度提升：** 分布式训练可以在多个GPU上并行进行计算，从而加速训练速度，特别是对于大型模型和数据集。

2. **处理大模型：** 如果计算图对单个GPU来说太大，分布式训练可以将模型拆分并放在多个GPU上，使之能够处理更大的模型。

3. **处理大数据集：** 分布式训练可以将数据分成多份，每个GPU上处理不同的数据子集，从而能够处理更大的数据集。

**缺点：**

1. **通信开销：** 在分布式训练中，需要在GPU之间进行梯度同步和参数更新，这可能会引入通信开销，影响训练速度。

2. **调试复杂性：** 分布式训练涉及到多个GPU和通信，可能会增加调试的复杂性。

3. **硬件要求：** 分布式训练需要多个GPU来并行执行计算，可能需要更多的硬件资源。

4. **算法选择：** 并不是所有模型和任务都适合分布式训练，某些情况下可能无法获得明显的加速效果。

总之，分布式训练可以在一定程度上加速大型模型的训练过程，但也需要权衡硬件资源、通信开销和调试复杂性等因素。